In [1]:
from keras.models import Sequential,load_model
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
df=pd.read_csv('celeba-dataset/list_attr_celeba.csv')
columns=df.columns.tolist()[1:]
df.replace(-1, 0, inplace = True)

In [3]:
df.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [4]:
image_size = 100
batch_size = 256

In [11]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
dataframe=df[:162770],
directory="celeba-dataset/img_align_celeba",
x_col="image_id",
y_col=columns,
batch_size=batch_size,
seed=42,
shuffle=True,
class_mode="other",
target_size=(100,100))

Found 162770 images.


In [12]:
valid_generator=test_datagen.flow_from_dataframe(
dataframe=df[162770:182637],
directory="celeba-dataset/img_align_celeba",
x_col="image_id",
y_col=columns,
batch_size=batch_size,
seed=42,
shuffle=True,
class_mode="other",
target_size=(100,100))

Found 19867 images.


In [8]:
test_generator=test_datagen.flow_from_dataframe(
dataframe=df[182637:],
directory="celeba-dataset/img_align_celeba",
x_col="image_id",
y_col=columns,
batch_size=1,
seed=42,
shuffle=False,
class_mode="other",
target_size=(100,100))

Found 19962 images.


In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(40, activation='sigmoid'))
model.compile(optimizers.rmsprop(lr=0.001, decay=1e-6),loss="binary_crossentropy",metrics=["accuracy"])

In [8]:
model = load_model('CelebFaces.h5')

In [15]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100
)

Epoch 1/100
635/635 [==============================] - 317s 500ms/step - loss: 0.3026 - acc: 0.8696 - val_loss: 0.2398 - val_acc: 0.8962
Epoch 2/100
635/635 [==============================] - 275s 434ms/step - loss: 0.2495 - acc: 0.8911 - val_loss: 0.2214 - val_acc: 0.9036
Epoch 3/100
635/635 [==============================] - 276s 435ms/step - loss: 0.2352 - acc: 0.8972 - val_loss: 0.2159 - val_acc: 0.9063
Epoch 4/100
635/635 [==============================] - 276s 435ms/step - loss: 0.2265 - acc: 0.9010 - val_loss: 0.2128 - val_acc: 0.9069
Epoch 5/100
635/635 [==============================] - 276s 434ms/step - loss: 0.2205 - acc: 0.9034 - val_loss: 0.2100 - val_acc: 0.9087
Epoch 6/100
635/635 [==============================] - 277s 436ms/step - loss: 0.2159 - acc: 0.9055 - val_loss: 0.2079 - val_acc: 0.9104
Epoch 7/100
635/635 [==============================] - 276s 434ms/step - loss: 0.2120 - acc: 0.9072 - val_loss: 0.2061 - val_acc: 0.9102
Epoch 8/100
635/635 [====================

In [16]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

19962/19962 [==============================] - 365s 18ms/step


In [17]:
pred_bool = (pred >0.5)

In [18]:
predictions = pred_bool.astype(int)
results=pd.DataFrame(predictions, columns=columns)
results["FileName"]=test_generator.filenames
ordered_cols=["FileName"]+columns
results=results[ordered_cols]#To get the same column order
results.to_csv("celebaFaces_result.csv",index=False)

In [19]:
model.save('CelebaFaces.h5')

In [20]:
test_generator.reset()
test_loss, test_acc = model.evaluate_generator(test_generator, steps = STEP_SIZE_TEST)
print('test acc:', test_acc)

test acc: 0.9043294754352948
